In [1]:
# Importing neccessary libraries
import tensorflow as tf
import tensorflow.keras as keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
import deeplake as dl
import time

c:\Users\etito\miniconda3\envs\tf-env\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.3.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
# Loading in Datasets
train_ds = dl.load('hub://activeloop/gtsrb-train')
test_ds = dl.load('hub://activeloop/gtsrb-test')
print(train_ds.summary())
train_shape = train_ds.images[0].shape
print(f"Image Shape: {train_shape}")

\

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtsrb-train



/

...

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtsrb-test



/

hub://activeloop/gtsrb-test loaded successfully.



Dataset(path='hub://activeloop/gtsrb-train', read_only=True, tensors=['images', 'boxes', 'labels', 'shapes', 'colors'])

 tensor      htype               shape              dtype  compression
 -------    -------             -------            -------  ------- 
 images      image     (39209, 25:225, 25:243, 3)   uint8    jpeg   
  boxes      bbox            (39209, 1, 4)         float32   None   
 labels   class_label          (39209, 1)          uint32    None   
 shapes   class_label          (39209, 1)          uint32    None   
 colors   class_label          (39209, 1)          uint32    None   
None
Image Shape: (26, 26, 3)


In [ ]:
# Resizing Images
def resize_images(images):
    resized = tf.image.resize(images, (26, 26)).numpy()
    normalized = resized / 255.0
    return normalized
train_images, test_images = train_ds.images.numpy(aslist=True), test_ds.images.numpy(aslist=True)
train_labels, test_labels = train_ds.labels.numpy().squeeze(), test_ds.labels.numpy().squeeze()

train_images_resized = np.array([resize_images(img) for img in train_images])
test_images_resized = np.array([resize_images(img) for img in test_images])

In [17]:
# Create and Train Model
def create_train_model(images, labels):
    model = keras.Sequential([
        layers.Input(shape=(26, 26, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(43, activation='softmax')
    ])
    lr_scheduler = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=0.001,
        decay_steps=10000,
        alpha=0.0)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_scheduler),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(images, labels, epochs=15, batch_size=128)
    return model
traffic_sign_model = create_train_model(train_images_resized, train_labels)

Epoch 1/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7394 - loss: 1.0576
Epoch 2/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9628 - loss: 0.1822
Epoch 3/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9837 - loss: 0.0813
Epoch 4/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9914 - loss: 0.0461
Epoch 5/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9949 - loss: 0.0290
Epoch 6/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9970 - loss: 0.0199
Epoch 7/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9983 - loss: 0.0132
Epoch 8/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9986 - loss: 0.0100
Epoch 9/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9995 - loss: 0.0071
Epoch 10/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9995 - loss: 0.0051
Epoch 11/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9998 - loss: 0.0039
Epoch 12/15
307/307 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
# Evaluating Model on Accuracy and Loss
def evaluate_model(model, images, labels):
    loss, accuracy = model.evaluate(images, labels)
    loss, accuracy = np.round(loss * 100, 2), np.round(accuracy * 100, 2)
    print(f"Loss: {loss} Accuracy: {accuracy}")
evaluate_model(traffic_sign_model, test_images_resized, test_labels)

230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9376 - loss: 0.2861
Loss: 28.61 Accuracy: 93.76


In [20]:
# Evaluating Latency Data
def evaluate_latency(model, images, num_samples=500):
    warmup_images = images[:10]
    measurement_images = images[10:10 + num_samples]
    for image in warmup_images:
        _ = model.predict(np.expand_dims(image, axis=0), verbose=0)
    time_records = []
    for image in measurement_images:
        sample = np.expand_dims(image, axis=0)
        start_time = time.monotonic()
        _ = model.predict(sample, verbose=0)
        end_time = time.monotonic()
        inference_time = (end_time - start_time) * 1000
        time_records.append(inference_time)
    return time_records
latency_records = evaluate_latency(traffic_sign_model, test_images_resized)

# Statistical Analysis of Latency
print(f"Average Latency: {np.round(np.mean(latency_records), 2)} ms")
print(f"Min Latency: {np.round(np.min(latency_records), 2)} ms")
print(f"Max Latency: {np.round(np.max(latency_records), 2)} ms")
print(f"Standard Deviation: {np.round(np.std(latency_records), 2)} ms")
print(f"Median Latency: {np.round(np.median(latency_records), 2)} ms")
print(f"99th Percentile Latency: {np.round(np.percentile(latency_records, 99), 2)} ms")
print(f"90th Percentile Latency: {np.round(np.percentile(latency_records, 90), 2)} ms")

Average Latency: 53.63 ms
Min Latency: 31.0 ms
Max Latency: 79.0 ms
Standard Deviation: 9.39 ms
Median Latency: 47.0 ms
99th Percentile Latency: 78.0 ms
90th Percentile Latency: 63.0 ms
